In [2]:
import import_ipynb

In [176]:
%run requirements.ipynb
%run config.ipynb
%run module_preprocess.ipynb
%run module_feature_extraction.ipynb
%run module_training.ipynb

In [89]:
four_time = pd.read_csv("CSV/four_time.csv")
v_value = pd.read_csv("CSV/v_value.csv")
mic = pd.read_csv("CSV/mic.csv")
dwt_distance_48 = pd.read_csv("CSV/tsc_dtw_sdtw_10_48.csv", names=['meter_dtw_48', 'meter_sdtw_48'], header=0)
dwt_distance_530 = pd.read_csv("CSV/tsc_dtw_sdtw_10_530_48.csv", names=['meter_dtw_530', 'meter_sdtw_530'], header=0)
# smart_meter = pd.read_csv("CSV/smart_meter.csv")
# fdi = pd.read_csv("CSV/FDI.csv")
print(four_time.shape, v_value.shape, mic.shape, dwt_distance_48.shape, dwt_distance_530.shape)

(2239250, 7) (2239250, 9) (2239250, 4) (2239250, 2) (2239250, 2)


In [70]:
dwt_distance_48.columns, dwt_distance_530.columns

(Index(['meter_dtw_48', 'meter_sdtw_48'], dtype='object'),
 Index(['meter_dtw_530', 'meter_sdtw_530'], dtype='object'))

In [73]:
dwt_distance_48.head(), dwt_distance_48.dtypes

(   meter_dtw_48  meter_sdtw_48
 0      2.524154     -68.984877
 1      1.896432     -71.081359
 2      1.963119     -71.260790
 3      2.880804     -64.383998
 4      2.117505     -69.355208,
 meter_dtw_48     float64
 meter_sdtw_48    float64
 dtype: object)

## prepare data

In [74]:
training_data = concat_two_df_and_drop_same_column_in_df2(four_time, v_value)
training_data = concat_two_df_and_drop_same_column_in_df2(training_data, mic)
training_data = concat_two_df_and_drop_same_column_in_df2(training_data, dwt_distance_48)
training_data = concat_two_df_and_drop_same_column_in_df2(training_data, dwt_distance_530)

,MeterID,Day,FDI,overnight,breakfast,daytime,evening
0,1236,2009-07-15,True,0.12852,0.0678,0.120048,0.119347


,MeterID,Day,FDI,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1
0,1236,2009-07-15,True,12,30,15,0.205161,0.209875,0.059876


,MeterID,Day,FDI,overnight,breakfast,daytime,evening,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1
0,1236,2009-07-15,True,0.12852,0.0678,0.120048,0.119347,12,30,15,0.205161,0.209875,0.059876


,MeterID,Day,FDI,overnight,breakfast,daytime,evening,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1
0,1236,2009-07-15,True,0.12852,0.0678,0.120048,0.119347,12,30,15,0.205161,0.209875,0.059876


,MeterID,Day,FDI,mic
0,1236,2009-07-15,True,0.624128


...

,meter_dtw_48,meter_sdtw_48
0,2.524154,-68.984877


,MeterID,Day,FDI,overnight,breakfast,daytime,evening,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1,mic,meter_dtw_48,meter_sdtw_48
0,1236,2009-07-15,True,0.12852,0.0678,0.120048,0.119347,12,30,15,0.205161,0.209875,0.059876,0.624128,2.524154,-68.984877


,MeterID,Day,FDI,overnight,breakfast,daytime,evening,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1,mic,meter_dtw_48,meter_sdtw_48
0,1236,2009-07-15,True,0.12852,0.0678,0.120048,0.119347,12,30,15,0.205161,0.209875,0.059876,0.624128,2.524154,-68.984877


,meter_dtw_530,meter_sdtw_530
0,3.991317,-50.432597


,MeterID,Day,FDI,overnight,breakfast,daytime,evening,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1,mic,meter_dtw_48,meter_sdtw_48,meter_dtw_530,meter_sdtw_530
0,1236,2009-07-15,True,0.12852,0.0678,0.120048,0.119347,12,30,15,0.205161,0.209875,0.059876,0.624128,2.524154,-68.984877,3.991317,-50.432597


In [18]:
training_data.head(3)

,MeterID,Day,FDI,overnight,breakfast,daytime,evening,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1,mic,meter_dtw,meter_sdtw
0,1236,2009-07-15,True,0.128520,0.06780,0.120048,0.119347,12,30,15,0.205161,0.209875,0.059876,0.624128,2.524154,-68.984877
1,1236,2009-07-16,True,0.084015,0.23875,0.201571,0.196692,20,34,24,0.656000,0.706000,0.012000,0.000000,1.896432,-71.081359
2,1236,2009-07-17,True,0.135647,0.17500,0.231000,0.408409,18,35,24,0.925000,1.387000,0.020000,0.000000,1.963119,-71.260790


## classifier

In [174]:
xgboost = XGBClassifier(scale_pos_weight=1, use_label_encoder=False, n_jobs=6, verbosity=0)
adaboost = AdaBoostClassifier()
lineSVM = LinearSVC(max_iter=1000000, C=50) # kernel = rbf
randforest = RandomForestClassifier(n_jobs=6)
complementNB = ComplementNB()
DT = DecisionTreeClassifier()
kNN = KNeighborsClassifier(n_jobs=6)

## sample training

In [122]:
x_col = ['overnight', 'breakfast', 'daytime', 'evening', 'Tmax1', 'Tmax2', 'Tmin1', 'Cmax1', 'Cmax2', 'Cmin1', 'mic', 'meter_dtw_48', 'meter_sdtw_48', 'meter_dtw_530', 'meter_sdtw_530']
y_col = ['FDI']
training_sample = training_data.iloc[:422*3*530]

#### xgboos whole

In [80]:
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 1, return_model=True)


 Time taken: 0 hours 2 minutes and 10.26 seconds.
average_precision_score: 0.9576343717904827
average_recall_score: 0.25132524707996406
average_accuracy_score: 0.92438075650541
average_roc_auc_score: 0.6250481746819879
average_f1_score: 0.39815671482456766


#### xgboost no 48

In [81]:
x_col = ['overnight', 'breakfast', 'daytime', 'evening', 'Tmax1', 'Tmax2', 'Tmin1', 'Cmax1', 'Cmax2', 'Cmin1', 'mic', 'meter_dtw_530', 'meter_sdtw_530']
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 3, return_model=True)


 Time taken: 0 hours 1 minutes and 37.97 seconds.

 Time taken: 0 hours 1 minutes and 39.39 seconds.

 Time taken: 0 hours 1 minutes and 43.58 seconds.
average_precision_score: 0.95528232067998
average_recall_score: 0.2450658879904163
average_accuracy_score: 0.9237235088974337
average_roc_auc_score: 0.6218994617045759
average_f1_score: 0.3900421986186518


#### xgboost no 530

In [83]:
x_col = ['overnight', 'breakfast', 'daytime', 'evening', 'Tmax1', 'Tmax2', 'Tmin1', 'Cmax1', 'Cmax2', 'Cmin1', 'mic', 'meter_dtw_48', 'meter_sdtw_48']
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 3, return_model=True)


 Time taken: 0 hours 1 minutes and 42.7 seconds.

 Time taken: 0 hours 1 minutes and 42.07 seconds.

 Time taken: 0 hours 1 minutes and 44.55 seconds.
average_precision_score: 0.9561944793054696
average_recall_score: 0.24276729559748425
average_accuracy_score: 0.9235290172583386
average_roc_auc_score: 0.6207691989407481
average_f1_score: 0.3872219081312886


#### xgboost whole weight=10

In [84]:
x_col = ['overnight', 'breakfast', 'daytime', 'evening', 'Tmax1', 'Tmax2', 'Tmin1', 'Cmax1', 'Cmax2', 'Cmin1', 'mic', 'meter_dtw_48', 'meter_sdtw_48', 'meter_dtw_530', 'meter_sdtw_530']
xgboost = XGBClassifier(scale_pos_weight=10, use_label_encoder=False, n_jobs=-1, verbosity=0)
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 3, return_model=True)


 Time taken: 0 hours 1 minutes and 59.43 seconds.

 Time taken: 0 hours 1 minutes and 56.29 seconds.

 Time taken: 0 hours 2 minutes and 1.51 seconds.
average_precision_score: 0.19155439658425402
average_recall_score: 0.6483453129679545
average_accuracy_score: 0.692659244686876
average_roc_auc_score: 0.6729512066329345
average_f1_score: 0.2957307508124474


#### four_time

In [124]:
x_col = ['overnight', 'breakfast', 'daytime', 'evening']
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 1, return_model=True)


 Time taken: 0 hours 2 minutes and 37.31 seconds.
average_precision_score: 0.9196428571428571
average_recall_score: 0.08560197663971249
average_accuracy_score: 0.9082491281409282
average_roc_auc_score: 0.5423876318153875
average_f1_score: 0.15662502054907115


#### v_value

In [125]:
x_col = [ 'Tmax1', 'Tmax2', 'Tmin1', 'Cmax1', 'Cmax2', 'Cmin1']
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 1, return_model=True)


 Time taken: 0 hours 3 minutes and 10.45 seconds.
average_precision_score: 0.9199613424705676
average_recall_score: 0.2351976639712489
average_accuracy_score: 0.9218456585889296
average_roc_auc_score: 0.6164679978247506
average_f1_score: 0.3746198704876391


#### mic

In [126]:
x_col = ['mic']
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 1, return_model=True)


 Time taken: 0 hours 1 minutes and 3.49 seconds.
average_precision_score: 0.9907002958996759
average_recall_score: 0.15792902066486972
average_accuracy_score: 0.9160444424573012
average_roc_auc_score: 0.5788825838180357
average_f1_score: 0.2724296258984443


#### tsc_48

In [127]:
x_col = ['meter_dtw_48', 'meter_sdtw_48']
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 1, return_model=True)


 Time taken: 0 hours 2 minutes and 39.95 seconds.
average_precision_score: 0.6399317406143344
average_recall_score: 0.008423180592991913
average_accuracy_score: 0.9008405615666637
average_roc_auc_score: 0.5039496737125834
average_f1_score: 0.016627499667450005


#### tsc_530

In [128]:
x_col = ['meter_dtw_530', 'meter_sdtw_530']
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 1, return_model=True)


 Time taken: 0 hours 2 minutes and 34.99 seconds.


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


average_precision_score: 0.0
average_recall_score: 0.0
average_accuracy_score: 0.9004739336492891
average_roc_auc_score: 0.5
average_f1_score: 0.0


#### tsc 48 & 530

In [177]:
x_col = ['meter_dtw_48', 'meter_sdtw_48', 'meter_dtw_530', 'meter_sdtw_530']
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 1, return_model=True)


 Time taken: 0 hours 1 minutes and 3.61 seconds.
average_precision_score: 0.690793283149971
average_recall_score: 0.035729260257562144
average_accuracy_score: 0.9025536824085445
average_roc_auc_score: 0.5169819782060379
average_f1_score: 0.06794430048124839


In [130]:
x_col = ['meter_dtw_48', 'meter_sdtw_48', 'meter_dtw_530', 'meter_sdtw_530']
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 1, return_model=True)


 Time taken: 0 hours 3 minutes and 19.53 seconds.
average_precision_score: 0.7030109489051095
average_recall_score: 0.0346136567834681
average_accuracy_score: 0.9024635607618707
average_roc_auc_score: 0.5164987350451601
average_f1_score: 0.06597876348689845


## SMOTE

In [134]:
# Oversample with SMOTE and random undersample for imbalanced dataset
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from matplotlib import pyplot
from numpy import where

In [160]:
Counter(training_data['FDI'])

Counter({True: 222600, False: 2016650})

In [161]:
x_col = ['overnight', 'breakfast', 'daytime', 'evening', 'Tmax1', 'Tmax2', 'Tmin1', 'Cmax1', 'Cmax2', 'Cmin1', 'mic', 'meter_dtw_48', 'meter_sdtw_48', 'meter_dtw_530', 'meter_sdtw_530']

In [162]:
oversample = SMOTE()
oversample_X, oversample_y = oversample.fit_resample(training_data[x_col], training_data['FDI'])

In [165]:
Counter(oversample_y)

Counter({True: 2016650, False: 2016650})

In [166]:
oversample_y = pd.DataFrame(oversample_y, columns=['FDI'])

In [167]:
smote_data = concat_two_df_and_drop_same_column_in_df2(oversample_X, oversample_y)

,overnight,breakfast,daytime,evening,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1,mic,meter_dtw_48,meter_sdtw_48,meter_dtw_530,meter_sdtw_530
0,0.12852,0.0678,0.120048,0.119347,12,30,15,0.205161,0.209875,0.059876,0.624128,2.524154,-68.984877,3.991317,-50.432597


,FDI
0,True


,overnight,breakfast,daytime,evening,Tmax1,Tmax2,Tmin1,Cmax1,Cmax2,Cmin1,mic,meter_dtw_48,meter_sdtw_48,meter_dtw_530,meter_sdtw_530,FDI
0,0.12852,0.0678,0.120048,0.119347,12,30,15,0.205161,0.209875,0.059876,0.624128,2.524154,-68.984877,3.991317,-50.432597,True


In [159]:
# smote on training_sample
model, scoe = train_single_model(xgboost, smote_data, x_col, y_col, split_num = 1, return_model=True)


 Time taken: 0 hours 2 minutes and 10.96 seconds.
average_precision_score: 0.8470510935609362
average_recall_score: 0.7183438155136268
average_accuracy_score: 0.7943175548935231
average_roc_auc_score: 0.7943175548935231
average_f1_score: 0.7774062774271743


In [182]:
x_col = ['overnight', 'breakfast', 'daytime', 'evening', 'Tmax1', 'Tmax2', 'Tmin1', 'Cmax1', 'Cmax2', 'Cmin1', 'mic', 'meter_dtw_48', 'meter_sdtw_48', 'meter_dtw_530', 'meter_sdtw_530']
# smote on all data
model, scoe = train_single_model(xgboost, smote_data, x_col, y_col, split_num = 3, return_model=True)

# imbalanced data 
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 3, return_model=True)


 Time taken: 0 hours 6 minutes and 56.95 seconds.

 Time taken: 0 hours 7 minutes and 1.75 seconds.

 Time taken: 0 hours 6 minutes and 54.53 seconds.
average_precision_score: 0.8459640082767406
average_recall_score: 0.7223315895172687
average_accuracy_score: 0.7954040942487127
average_roc_auc_score: 0.7954040942487128
average_f1_score: 0.7792693628677908

 Time taken: 0 hours 3 minutes and 25.23 seconds.

 Time taken: 0 hours 2 minutes and 58.47 seconds.

 Time taken: 0 hours 3 minutes and 24.26 seconds.
average_precision_score: 0.9560228141206603
average_recall_score: 0.24511829889188377
average_accuracy_score: 0.9238375944326598
average_roc_auc_score: 0.6219368302532212
average_f1_score: 0.39018849693571084


In [183]:
x_col = ['overnight', 'breakfast', 'daytime', 'evening', 'Tmax1', 'Tmax2', 'Tmin1', 'Cmax1', 'Cmax2', 'Cmin1', 'mic', 'meter_dtw_48', 'meter_sdtw_48', 'meter_dtw_530', 'meter_sdtw_530']
xgboost = XGBClassifier(scale_pos_weight=10, use_label_encoder=False, n_jobs=6, verbosity=0)
# smote on all data
model, scoe = train_single_model(xgboost, smote_data, x_col, y_col, split_num = 3, return_model=True)

# imbalanced data 
model, score = train_single_model(xgboost, training_data, x_col, y_col, split_num = 3, return_model=True)


 Time taken: 0 hours 6 minutes and 44.42 seconds.

 Time taken: 0 hours 7 minutes and 27.35 seconds.

 Time taken: 0 hours 7 minutes and 13.06 seconds.
average_precision_score: 0.5525945823224182
average_recall_score: 0.9934269429223933
average_accuracy_score: 0.5945393488100452
average_roc_auc_score: 0.5945393488100452
average_f1_score: 0.7101589543269068

 Time taken: 0 hours 4 minutes and 14.14 seconds.

 Time taken: 0 hours 3 minutes and 38.09 seconds.

 Time taken: 0 hours 3 minutes and 31.95 seconds.
average_precision_score: 0.19184276099907524
average_recall_score: 0.6470200658879904
average_accuracy_score: 0.6939386947514669
average_roc_auc_score: 0.6730688447248089
average_f1_score: 0.2959347416254148


In [184]:
# smote on all data adaboost
model, scoe = train_single_model(adaboost, smote_data, x_col, y_col, split_num = 3, return_model=True)
# imbalanced data adaboost
model, score = train_single_model(adaboost, training_data, x_col, y_col, split_num = 3, return_model=True)


 Time taken: 0 hours 10 minutes and 59.17 seconds.

 Time taken: 0 hours 11 minutes and 8.46 seconds.

 Time taken: 0 hours 10 minutes and 58.65 seconds.
average_precision_score: 0.6971849991309447
average_recall_score: 0.5112014699846005
average_accuracy_score: 0.6445127645683022
average_roc_auc_score: 0.6445127645683022
average_f1_score: 0.5898345754145229

 Time taken: 0 hours 5 minutes and 22.34 seconds.

 Time taken: 0 hours 4 minutes and 14.73 seconds.

 Time taken: 0 hours 5 minutes and 4.94 seconds.
average_precision_score: 0.867681902535495
average_recall_score: 0.0678995707297594
average_accuracy_score: 0.9063123813776933
average_roc_auc_score: 0.533378430780682
average_f1_score: 0.12594264992418466


## XGBoost